# Approximate Nearest Neighbors

In [ ]:
import numpy as np
import pandas as pd

from spark_rapids_ml.knn import ApproximateNearestNeighbors
from pyspark.sql.functions import col

### Create synthetic dataset

In [ ]:
dim = 2000
dtype = 'float32'
np.random.seed(1)

# items
num_vecs = 100000
vec = np.random.randn(dim).reshape([1,dim])
arr = np.random.randn(num_vecs).reshape([num_vecs,1])
items = arr * vec
items = items.astype(dtype)

# items extra data
items_extra = np.random.randn(num_vecs)

# queries
num_vecs = 50
vec = np.random.randn(dim).reshape([1,dim])
arr = np.random.randn(num_vecs).reshape([num_vecs,1])
queries = arr * vec
queries = queries.astype(dtype)

# queries extra data
queries_extra = np.random.randn(num_vecs)

### Configure Spark
It is highly recommend to increase the spark.sql.execution.arrow.maxRecordsPerBatch from the default 10000 to a larger value. Spark Rapids ML applies cuML approximate nearest neighbor search on every data batch independently, and some algorithms have requirements on the batch size. For example, the ivfflat algorithm requires that the number of vectors in a batch must be more than the number of kmeans centroids (specified by 'nlist').

In [ ]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 0) # set to unlimited

### Convert dataset to Spark DataFrame

In [ ]:
pd_items = pd.DataFrame({"features": list(items), "extra": items_extra})
item_df = spark.createDataFrame(pd_items, "features array<float>, extra float")

pd_queries = pd.DataFrame({"features": list(queries), "extra": queries_extra})
query_df = spark.createDataFrame(pd_queries, "features array<float>, extra float")

In [ ]:
item_df.show(5, truncate=80)

In [ ]:
query_df.show(5, truncate=80)

## Spark RAPIDS ML (GPU)
The ApproximateNearestNeighbors class of Spark Rapids ML uses the ivfflat algorithm by default.

In [ ]:
knn = ApproximateNearestNeighbors(k=2)
knn.setInputCol("features")

In [ ]:
knn_model = knn.fit(item_df)

Note: `fit` just stores a reference to the `item_df` in the returned model.  As such, saving the estimator or model is not supported, since their only state is the referenced dataset.  Instead, just re-create and re-fit the estimator on the dataset, as needed.

#### kneighbors

This API takes a DataFrame of query vectors, and returns the `k` approximate nearest item vectors for each query vector, represented by their unique ids and distances.  The unique ids are automatically generated if not provided, so the input datasets are also returned with their unique ids.

In [ ]:
item_id_df, query_id_df, neighbor_df = knn_model.kneighbors(query_df)

In [ ]:
# original item_df is returned with unique identifiers
item_id_df.show(5, truncate=80)

In [ ]:
# original query_df is returned with unique identifiers
query_id_df.show(5, truncate=80)

In [ ]:
# neighbor_df shows the nearest item vectors for each query vector, represented by their unique ids and distances.
neighbor_df.show()

In [ ]:
# change the value of 'k'
knn_model.setK(3)
_, _, neighbor_df = knn_model.kneighbors(query_df)

In [ ]:
neighbor_df.show()

#### approxNearestNeighborsJoin

This API returns a join of the query vectors and their `k` approximate nearest item vectors.

In [ ]:
result_df = knn_model.approxSimilarityJoin(query_df)

In [ ]:
result_df.orderBy("query_df", "item_df").show()

For each returned query or item vector, all columns from the original input DataFrame will be returned as a single struct column.

In [ ]:
result_df.select("query_df.*").show()

## PySpark

PySpark has an LSH-based Approximate Nearest Neighbors implementation, shown here to illustrate the similarity between the APIs.  However, the algorithms are very different, so their results are only roughly comparable, and it would require elaborate tuning of parameters to produce similar results.

In [ ]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.functions import array_to_vector
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

item_vector_df = item_df.select(array_to_vector(item_df.features).alias("features"))
query_vector_df = query_df.select(array_to_vector(query_df.features).alias("features"))
key = Vectors.dense([1.0] * dim)

In [ ]:
item_vector_df.show(5, truncate=80)

In [ ]:
query_vector_df.show(5, truncate=80)

In [ ]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0, numHashTables=3)
model = brp.fit(item_vector_df)

In [ ]:
# Feature Transformation
print("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(item_vector_df).show(5)

In [ ]:
# Compute the locality sensitive hashes for the input rows, then perform approximate similarity join.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxSimilarityJoin(transformed_item_vector_df, transformed_query_vector_df, 3.0)`
print("Approximately joining items and queries on Euclidean distance smaller than 3.0:")
model.approxSimilarityJoin(item_vector_df, query_vector_df, 3.0, distCol="EuclideanDistance")\
    .select(col("datasetA.features").alias("item"),
            col("datasetB.features").alias("query"),
            col("EuclideanDistance")).orderBy("query", "item").show()

In [ ]:
# Compute the locality sensitive hashes for the input rows, then perform approximate nearest neighbor search.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxNearestNeighbors(transformed_item_vector_df, key, 2)`
print("Approximately searching item vectors for 2 nearest neighbors of the key:")
model.approxNearestNeighbors(item_vector_df, key, 2).show()

In [ ]:
# saves the LSH hashes for the input rows
model.write().overwrite().save("/tmp/ann_model")

## Spark Rapids ML (GPU CAGRA algorithm) 
CAGRA is a cutting-edge graph-based algorithm available in cuVS, and is now integrated into the ApproximateNearestNeighbors class of Spark Rapids ML. Cagra currently supports sqeuclidean distance metric only, and the metric must be set before using the main APIs.

In [ ]:
knn = ApproximateNearestNeighbors(k=2, algorithm='cagra', metric='sqeuclidean', algoParams={"build_algo" : "nn_descent"})
knn.setInputCol("features")
knn_model = knn.fit(item_df)
item_id_df, query_id_df, neighbor_df = knn_model.kneighbors(query_df)
neighbor_df.show()

In [ ]:
result_df = knn_model.approxSimilarityJoin(query_df)
result_df.show()

## Spark Rapids ML (GPU IVFPQ algorithm)
The IVFPQ algorithm combines the power of Inverted File Indexing with Product Quantization to deliver fast and memory-efficient approximate nearest neighbor search. It is now integrated into Spark Rapids ML.

In [ ]:
knn = ApproximateNearestNeighbors(k=2, algorithm='ivfpq', algoParams={"M": 2, "n_bits": 8})
knn.setInputCol("features")
knn_model = knn.fit(item_df)
item_id_df, query_id_df, neighbor_df = knn_model.kneighbors(query_df)
neighbor_df.show()

In [ ]:
result_df = knn_model.approxSimilarityJoin(query_df)
result_df.show()